In [7]:
import random
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

from  buffer import ReplayBuffer

In [131]:
PASS = 0
BET = 1
NUM_ACTIONS = 2
NUM_PLAYERS = 2
nodeMap = {}

regretNet = keras.Sequential([
    keras.layers.Normalization(input_shape=[12, ]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(2)
])

regretNet.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

strategyNet = keras.models.clone_model(regretNet)

def getStrategy(infoSet):
    encoder = KuhnEncoder()
    regrets = tf.reshape(regretNet(encoder.encode(infoSet)), (NUM_ACTIONS,))
    regrets = tf.nn.relu(regrets)
    strategy, regretSum = tf.linalg.normalize(regrets, ord=1)
    if regretSum < 0.01:
        strategy, _ = tf.linalg.normalize(np.ones(tf.shape(strategy), dtype=np.float32), ord=1)
    return strategy
    

def getAverageStrategy(infoSet):
    encoder = KuhnEncoder()
    regrets = tf.reshape(strategyNet(encoder.encode(infoSet)), (NUM_ACTIONS,))
    regret_sum = tf.sum(regrets)
    

def getSample(strategy):
    cum_probs = tf.cumsum(strategy)
    rand_num = tf.random.uniform([], 0, 1)
    i = tf.argmax(cum_probs > rand_num)
    return i


class KuhnEncoder():
    def __init__(self):
        self.infostates = pd.Series(["1", "2", "3", "1p", "2p", "3p", 
                                     "1b", "2b", "3b", "1pb", "2pb", "3pb"])
    
    def encode(self, infoState):
        return np.array(self.infostates == infoState).astype(float)


def IsTerminal(cards, history, player):
    plays = len(history)
    opponent = 1 - player
    if plays > 1 :
        terminalPass = history[-1] == "p"
        doubleBet = history[-2:] == "bb"
        isPlayerCardHigher = cards[player] > cards[opponent]
        if terminalPass:
            if history == "pp":
                return 1 if isPlayerCardHigher else -1
            else:
                return 1
        elif doubleBet:
            return 2 if isPlayerCardHigher else -2
    else:
        return None
    

In [132]:
def traverse(cards, history, traversingPlayer, time, valueBuffer, strategyBuffer):
    plays = len(history)
    player = plays % 2
    infoSet = str(cards[player]) + history
    strategy = getStrategy(infoSet)

    if(IsTerminal(cards, history, traversingPlayer) is not None):
        return IsTerminal(cards, history, traversingPlayer)
    elif player == traversingPlayer:
        nodeUtil = 0
        util = [0 for _ in range(NUM_ACTIONS)]
        regret = [0 for _ in range(NUM_ACTIONS)]
        for a in range(NUM_ACTIONS):
            nextHistory = history + ('p' if a == PASS else 'b')
            util[a] = traverse(cards, nextHistory, traversingPlayer, time, 
                valueBuffer, strategyBuffer)
            nodeUtil += util[a] * strategy[a]
        for a in range(NUM_ACTIONS):
            regret[a] = util[a] - nodeUtil
            valueBuffer.insert((infoSet, time, regret))
        print("Infoset:{}, Strategy:{}, Regrets:[{}, {}]".format(infoSet, strategy, regret[0], regret[1]))
        return nodeUtil
    else:
        strategyBuffer.insert((infoSet, time, strategy))
        a = getSample(strategy)
        nextHistory = history + ('p' if a == PASS else 'b')
        return traverse(cards, nextHistory, traversingPlayer, time, 
                        valueBuffer, strategyBuffer)


def train(inner_iterations, outer_iterations):
    print("\n\n\n")
    for i in range(outer_iterations):
        valBuffer = ReplayBuffer(30)
        stratBuffer = ReplayBuffer(30)
        for j in range(inner_iterations):
            cards = [1, 2, 3]
            random.shuffle(cards)
            
            for traversingPlayer in range(NUM_PLAYERS): 
                traverse(cards, '', traversingPlayer, j, 
                        valBuffer, stratBuffer)
            # train value net
        # train strategy net

    return valBuffer

In [80]:
def generateDataset(buffer):
    x, y = [], []
    for (i, t, sigma) in buffer.data:
        x.append(KuhnEncoder().encode(i))
        y.append(sigma)
    return np.array(x), np.array(y)

x, y = generateDataset(buffer)
df = pd.DataFrame(y)
df

,0,1
0,3.000000,0.000000
1,-1.000000,0.000000
2,0.000000,0.000000
3,-1.000000,0.000000
4,3.000000,0.000000
5,1.544516,-1.455484
6,3.000000,0.000000
7,3.000000,0.000000
8,0.000000,0.000000
9,0.000000,0.000000


In [35]:
df = pd.DataFrame(x)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
